<a href="https://colab.research.google.com/github/esintao/Konrad-og-Esin/blob/main/Chain_of_thought.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from datasets import load_dataset
from transformers import pipeline

# Load model

In [100]:
prompt = "Write an email about an alpaca that likes flan"

model = pipeline(task="text2text-generation", model="google/flan-t5-base")

model(prompt, max_length=128, do_sample=True)

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "I just got back from a trip to New York. I haven't been able to get a photo of the alpaca yet, but I'll let you know when I get back. She's a teddy bear and loves flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a teddy bear and likes flan. She's a"}]

# Load dataset

In [25]:
import pandas as pd

In [26]:
dataset = load_dataset("derek-thomas/ScienceQA")

df = pd.DataFrame.from_dict(dataset['train'])

df = df[(df['choices'].str.len() == 4) & (df['image'].isna())]

df = df.sample(n=200)



# Prompting without CoT

In [58]:
df['baseline_result'] = pd.NA
df['correct/incorrect'] = pd.NA



for i, row in df.iterrows():
    question_text = (
        f"Question: {row['question']}\n"
        f"(A) {row['choices'][0]} "
        f"(B) {row['choices'][1]} "
        f"(C) {row['choices'][2]} "
        f"(D) {row['choices'][3]}"
        f"The answer must be formatted as 1 letter, A-D."
    )

    baseline_result = model(question_text, max_new_tokens=5, temperature=0.1, do_sample=False)

    generated = baseline_result[0]['generated_text']

    df.loc[i, 'baseline_result'] = generated


df['predicted_letter'] = (df['baseline_result'].str.extract(r'\b([ABCD])\b', expand=False))

df['gold_letter'] = df['answer'].map({0: 'A', 1: 'B', 2: 'C', 3: 'D'})

df['correct/incorrect'] = (df['predicted_letter'] == df['gold_letter']).astype('Int64')


In [59]:
df

,image,question,choices,answer,hint,task,grade,subject,topic,category,skill,lecture,solution,baseline_result,correct/incorrect,predicted_letter,gold_letter,CoT_result
1310,None,What do these two changes have in common?\naci...,"[Both are caused by heating., Both are caused ...",3,,closed choice,grade8,natural science,chemistry,Chemical reactions,Compare physical and chemical changes,Matter is made of very small particles called ...,Step 1: Think about each change.\nAcid rain we...,D,1,D,D,Reasoning:The heart is the organ that produces...
5695,None,Which type of sentence is this?\nAs Colin sat ...,"[compound, complex, compound-complex, simple]",1,,closed choice,grade8,language science,grammar,Phrases and clauses,"Is the sentence simple, compound, complex, or ...",A clause is a group of words that contains bot...,The sentence is complex. It is made up of an i...,(A),0,A,B,Reasoning:The heart is the organ that produces...
1002,None,How long is a bench?,"[9 miles, 9 feet, 9 inches, 9 yards]",1,Select the best estimate.,closed choice,grade8,natural science,units-and-measurement,Units and measurement,"Choose customary units of distance, mass, and ...",Measurements are written with both a number an...,The best estimate for the length of a bench is...,B,1,B,B,Reasoning:The heart is the organ that produces...
458,None,Which organ uses acid to break down food?,"[brain, stomach, muscles, skin]",1,,closed choice,grade5,natural science,biology,Animals,Human organs and their functions,,,(B),1,B,B,Reasoning:The heart is the organ that produces...
2278,None,What do these two changes have in common?\nchi...,"[Both are caused by cooling., Both are caused ...",3,,closed choice,grade8,natural science,chemistry,Chemical reactions,Compare physical and chemical changes,Matter is made of very small particles called ...,Step 1: Think about each change.\nCooking chic...,(D),1,D,D,Reasoning:The heart is the organ that produces...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3407,None,Select the mammal.,"[penguin, keel-billed toucan, American bullfro...",3,,closed choice,grade5,natural science,biology,Classification,"Identify mammals, birds, fish, reptiles, and a...",Mammals have hair or fur and feed their young ...,A koala is a mammal. It has fur and feeds its ...,D,1,D,D,NaN
2248,None,How long is the Mississippi River?,"[2,300 feet, 2,300 inches, 2,300 yards, 2,300 ...",3,Select the best estimate.,closed choice,grade7,natural science,units-and-measurement,Units and measurement,"Choose customary units of distance, mass, and ...",Measurements are written with both a number an...,The best estimate for the length of the Missis...,D,1,D,D,NaN
9731,None,What do these two changes have in common?\nsal...,"[Both are chemical changes., Both are caused b...",0,,closed choice,grade7,natural science,chemistry,Chemical reactions,Compare physical and chemical changes,Matter is made of very small particles called ...,Step 1: Think about each change.\nSaliva break...,(A),1,A,A,NaN
2767,None,How long is a soccer field?,"[110 kilometers, 110 meters, 110 centimeters, ...",1,Select the best estimate.,closed choice,grade7,natural science,units-and-measurement,Units and measurement,"Choose metric units of distance, mass, and volume",Measurements are written with both a number an...,The best estimate for the length of a soccer f...,(B),1,B,B,NaN


# Prompting with CoT

In [ ]:
answers_CoT = []
question_list_CoT = []

# 1. Define a template with a "Few-Shot" example
FEW_SHOT_EXAMPLE = """You are a careful reasoning assistant that answers multiple-choice questions about science.
First, think step by step. Then, give the final answer.

Example Question: What is the main source of energy for Earth?
(A) The Moon (B) The Sun (C) Burning coal (D) Wind
Reasoning: Step 1: Earth's climate and life processes require a massive constant energy source.
Reasoning: Step 2: While wind and coal provide energy, they are secondary sources.
Reasoning: Step 3: The Sun provides the vast majority of energy to Earth via radiation.
Answer: B

---
"""


answers_CoT = []

for i, row in df.iterrows():
    # 2. Construct the prompt with clear newlines and the example
    prompt = (
            "You are a careful reasoning assistant that answers multiple-choice questions about science. "
            "First, think step by step. Then, give the final answer. "
            "Format exactly as:\n"
            "Reasoning: <step 1>\n"
            "Reasoning: <step 2>\n"
            "Reasoning: <step 3>\n"
            "Answer: <final letter>\n\n"
            f"Question: {row['question']}\n"
            f"Options: (A) {row['choices'][0]} (B) {row['choices'][1]} (C) {row['choices'][2]} (D) {row['choices'][3]}\n\n"
            "Reasoning:"
        )

    CoT_result = model(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, repetition_penalty=2.5)

    # We prepend "Reasoning:" back to the text since we used it as a nudge
    generated_text = "Reasoning:" + CoT_result[0]['generated_text']
    df.loc[i,'CoT_result'] = generated_text
    answers_CoT.append(generated_text)
    question_list_CoT.append(prompt)

pattern = r"Reasoning:\s*(.*?)\s*The final answer:\s*\(?([A-D])\)?"
df[['reasoning', 'predicted_letter_CoT']] = df['CoT_result'].str.extract(pattern)

df['correct/incorrect_CoT'] = (df['predicted_letter_CoT'] == df['gold_letter']).astype('Int64')



In [ ]:
df

# Flip analysis

In [ ]:


#flip = pd.DataFrame({'question': question_list, 'answer': answers, 'answer_CoT': answers_CoT, 'q_answer': df['answer'].apply(lambda x: ['A','B','C','D'][x-1])})

